1. regular classification - done
2. random columns classification - done
3. classification using PCA feature reduction - done
4. for dist in (wesserstain, hellinger, JM):
    1. ranked columns classification (for ranking, use 1st eigen vector from the diffusion maps)        
    2. ranked + K-means classification (for ranking, use 1st eigen vector from the diffusion maps)
    3. K-mediods classification
    4. K-means + pick feature by distance from axis (0, 0)
### we want to deploy over all distance functions!
    
* ask Neta if the ranking we wrote is correct
* Chen will write the function regarding the ranking values

classification
1. add cross validation k-folds (5-10 folds)
2. ranked + K-means: rank by mean row of the flatten matrix. (the bigger the better)


In [5]:
from math import exp, sqrt, log
import numpy as np
import pandas as pd
import seaborn as sns
from sklearn import preprocessing

from dictances import jensen_shannon
import matplotlib.pyplot as plt
from pydiffmap import diffusion_map as dm
from pydiffmap.visualization import embedding_plot, data_plot

from ref.diffusion_maps import diffusion_mapping
from ref.Shir import utils as shir_utils
from utils.machine_learning import min_max_scaler
from utils.general import calc_mean_std, flatten
from utils.distances import norm_by_dist_type, calculate_distance
#from main import execute_distance_func, calc_dist, export_heatmaps, k_medoids_features, return_best_features_by_kmeans
from sklearn.ensemble import RandomForestClassifier
from sklearn.multiclass import OneVsRestClassifier
from sklearn.model_selection import train_test_split, cross_val_score, GroupKFold, KFold, StratifiedKFold
from sklearn import metrics
from sklearn.decomposition import PCA
from sklearn.preprocessing import StandardScaler

In [6]:
def predict(X_train, X_test, y_train, y_test):
    kf = StratifiedKFold(n_splits=5, shuffle=True)
    clf = RandomForestClassifier(random_state=1)
    multi_target_forest = OneVsRestClassifier(clf, n_jobs=-1)
    train_acc = []
    
    for train_index, test_index in kf.split(X_train, y_train):
        model = multi_target_forest.fit(X_train.iloc[train_index], y_train.iloc[train_index])
        train_preds = model.predict(X_train.iloc[test_index])
        
        train_acc.append(metrics.accuracy_score(y_train.iloc[test_index], train_preds))        
    
    model = multi_target_forest.fit(X_train, y_train)
    preds = model.predict(X_test)
    print(metrics.classification_report(y_test, preds, digits=3))
    return train_acc

In [7]:
data = pd.read_csv('data/glass.csv')
features = data.columns.drop('label')
label_column = 'label'
k = 4

## Regular classification

In [8]:
X_train, X_test, y_train, y_test = train_test_split(data[features], data[label_column], test_size=0.33, random_state=42)
train_acc = predict(X_train, X_test, y_train, y_test)
train_avg_score = sum(train_acc)/len(train_acc)
print(f'{train_avg_score}')

C:\Users\chenb\anaconda3\lib\site-packages\sklearn\base.py:333: UserWarning: Trying to unpickle estimator DecisionTreeClassifier from version 0.23.2 when using version 1.0.1. This might lead to breaking code or invalid results. Use at your own risk. For more info please refer to:
https://scikit-learn.org/stable/modules/model_persistence.html#security-maintainability-limitations
  self.__class__.__name__, pickle_version, __version__),
C:\Users\chenb\anaconda3\lib\site-packages\sklearn\base.py:333: UserWarning: Trying to unpickle estimator RandomForestClassifier from version 0.23.2 when using version 1.0.1. This might lead to breaking code or invalid results. Use at your own risk. For more info please refer to:
https://scikit-learn.org/stable/modules/model_persistence.html#security-maintainability-limitations
  self.__class__.__name__, pickle_version, __version__),
C:\Users\chenb\anaconda3\lib\site-packages\sklearn\base.py:439: UserWarning: X has feature names, but RandomForestClassifier

C:\Users\chenb\anaconda3\lib\site-packages\sklearn\base.py:439: UserWarning: X has feature names, but RandomForestClassifier was fitted without feature names
  
C:\Users\chenb\anaconda3\lib\site-packages\sklearn\base.py:439: UserWarning: X has feature names, but RandomForestClassifier was fitted without feature names
  
C:\Users\chenb\anaconda3\lib\site-packages\sklearn\base.py:439: UserWarning: X has feature names, but RandomForestClassifier was fitted without feature names
  
C:\Users\chenb\anaconda3\lib\site-packages\sklearn\base.py:439: UserWarning: X has feature names, but RandomForestClassifier was fitted without feature names
  
C:\Users\chenb\anaconda3\lib\site-packages\sklearn\base.py:439: UserWarning: X has feature names, but RandomForestClassifier was fitted without feature names
  
C:\Users\chenb\anaconda3\lib\site-packages\sklearn\base.py:439: UserWarning: X has feature names, but RandomForestClassifier was fitted without feature names
  
C:\Users\chenb\anaconda3\lib\site-

              precision    recall  f1-score   support

           1      0.769     0.909     0.833        22
           2      0.792     0.760     0.776        25
           3      1.000     0.500     0.667         4
           5      1.000     0.500     0.667         6
           6      0.800     1.000     0.889         4
           7      0.909     1.000     0.952        10

    accuracy                          0.817        71
   macro avg      0.878     0.778     0.797        71
weighted avg      0.831     0.817     0.809        71

0.7623152709359605


C:\Users\chenb\anaconda3\lib\site-packages\sklearn\base.py:439: UserWarning: X has feature names, but RandomForestClassifier was fitted without feature names
  
C:\Users\chenb\anaconda3\lib\site-packages\sklearn\base.py:439: UserWarning: X has feature names, but RandomForestClassifier was fitted without feature names
  
C:\Users\chenb\anaconda3\lib\site-packages\sklearn\base.py:439: UserWarning: X has feature names, but RandomForestClassifier was fitted without feature names
  
C:\Users\chenb\anaconda3\lib\site-packages\sklearn\base.py:439: UserWarning: X has feature names, but RandomForestClassifier was fitted without feature names
  
C:\Users\chenb\anaconda3\lib\site-packages\sklearn\base.py:439: UserWarning: X has feature names, but RandomForestClassifier was fitted without feature names
  
C:\Users\chenb\anaconda3\lib\site-packages\sklearn\base.py:439: UserWarning: X has feature names, but RandomForestClassifier was fitted without feature names
  


In [13]:
#pip install skfeature-chappers
from skfeature.function.similarity_based import fisher_score

In [14]:
ranks= fisher_score.fisher_score(X_train,y_train)

ValueError: Length of passed values is 1, index implies 9.

## Random columns

In [79]:
sampled_data = data[features].sample(n=k, axis='columns')
new_features = sampled_data.columns
sampled_data[label_column] = data[label_column]
X_train, X_test, y_train, y_test = train_test_split(sampled_data[new_features], sampled_data[label_column], test_size=0.33, random_state=42)

train_acc = predict(X_train, X_test, y_train, y_test)
train_avg_score = sum(train_acc)/len(train_acc)
print(f'{train_avg_score}')

              precision    recall  f1-score   support

           1      0.633     0.864     0.731        22
           2      0.696     0.640     0.667        25
           3      0.000     0.000     0.000         4
           5      0.667     0.333     0.444         6
           6      0.800     1.000     0.889         4
           7      1.000     1.000     1.000        10

    accuracy                          0.718        71
   macro avg      0.633     0.639     0.622        71
weighted avg      0.683     0.718     0.690        71

0.6650246305418719


C:\Users\chenb\anaconda3\lib\site-packages\sklearn\metrics\_classification.py:1308: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
C:\Users\chenb\anaconda3\lib\site-packages\sklearn\metrics\_classification.py:1308: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
C:\Users\chenb\anaconda3\lib\site-packages\sklearn\metrics\_classification.py:1308: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


## PCA Classification

In [80]:
X_train, X_test, y_train, y_test = train_test_split(data[features], data[label_column], test_size=0.33, random_state=42)
# Norm
X_train_norm, X_test_norm = min_max_scaler(X_train, features, X_test, False)

#PCA
pca = PCA(n_components=k)
pca.fit(X_train_norm)
X_train_pca = pca.transform(X_train_norm)
X_test_pca = pca.transform(X_test_norm)
y_train_arr = y_train.to_numpy()
y_test_arr = y_test.to_numpy()



In [81]:
def predict_np(X_train, X_test, y_train, y_test):
    kf = StratifiedKFold(n_splits=5, shuffle=True)
    clf = RandomForestClassifier(random_state=1)
    multi_target_forest = OneVsRestClassifier(clf, n_jobs=-1)
    train_acc = []
    
    for train_index, test_index in kf.split(X_train, y_train):
        model = multi_target_forest.fit(X_train[train_index], y_train[train_index])
        train_preds = model.predict(X_train[test_index])
        
        train_acc.append(metrics.accuracy_score(y_train[test_index], train_preds))        
    
    model = multi_target_forest.fit(X_train, y_train)
    preds = model.predict(X_test)
    print(metrics.classification_report(y_test, preds, digits=3))
    return train_acc

In [82]:
train_acc = predict_np(X_train_pca, X_test_pca, y_train_arr, y_test_arr)
train_avg_score = sum(train_acc)/len(train_acc)
print(f'{train_avg_score}')

              precision    recall  f1-score   support

           1      0.720     0.818     0.766        22
           2      0.643     0.720     0.679        25
           3      0.000     0.000     0.000         4
           5      0.600     0.500     0.545         6
           6      1.000     0.500     0.667         4
           7      0.818     0.900     0.857        10

    accuracy                          0.704        71
   macro avg      0.630     0.573     0.586        71
weighted avg      0.672     0.704     0.681        71

0.6568965517241379


C:\Users\chenb\anaconda3\lib\site-packages\sklearn\metrics\_classification.py:1308: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
C:\Users\chenb\anaconda3\lib\site-packages\sklearn\metrics\_classification.py:1308: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
C:\Users\chenb\anaconda3\lib\site-packages\sklearn\metrics\_classification.py:1308: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


# Our Methodology

In [83]:
# df_norm = min_max_scaler(df, features)

## Wasserstein Distance

In [84]:
X_train, X_test, y_train, y_test = train_test_split(data[features], data[label_column], test_size=0.33, random_state=42)
# Norm
X_train_norm, X_test_norm = min_max_scaler(X_train, features, X_test)

df_dists, dist_dict = calc_dist('wasserstein_dist', X_train_norm, y_train)

eps_type='maxmin'#mean' #or maxmin
alpha=1
vec, egs, coordinates, dataList, epsilon, ranking = (diffusion_mapping(df_dists, alpha, eps_type, 8, 1, dim=2))

### pick feature by distance from axis (0, 0)

In [85]:
best_features = return_farest_featires_from_center(k, coordinates)
print(f'{best_features}')
predict(X_train.iloc[:, best_features], X_test.iloc[:, best_features], y_train, y_test)

[6 8 0 2]
              precision    recall  f1-score   support

           1      0.692     0.818     0.750        22
           2      0.714     0.600     0.652        25
           3      0.000     0.000     0.000         4
           5      0.333     0.333     0.333         6
           6      0.375     0.750     0.500         4
           7      1.000     0.900     0.947        10

    accuracy                          0.662        71
   macro avg      0.519     0.567     0.530        71
weighted avg      0.656     0.662     0.652        71



[0.7241379310344828,
 0.7931034482758621,
 0.6896551724137931,
 0.75,
 0.7857142857142857]

### ranked columns classification (for ranking, use 1st eigen vector from the diffusion maps)

In [86]:
flat_ranking = [item for sublist in ranking for item in sublist]
ranking_idx = np.argsort(flat_ranking)
print(f'{ranking_idx}')
predict(X_train.iloc[:, ranking_idx[-k:]], X_test.iloc[:, ranking_idx[-k:]], y_train, y_test)

[2 3 7 1 4 5 6 8 0]
              precision    recall  f1-score   support

           1      0.655     0.864     0.745        22
           2      0.682     0.600     0.638        25
           3      0.000     0.000     0.000         4
           5      0.400     0.333     0.364         6
           6      0.800     1.000     0.889         4
           7      1.000     0.800     0.889        10

    accuracy                          0.676        71
   macro avg      0.589     0.599     0.587        71
weighted avg      0.663     0.676     0.662        71



[0.4827586206896552,
 0.7241379310344828,
 0.7931034482758621,
 0.6785714285714286,
 0.7857142857142857]

### ranked + K-means classification (for ranking, use 1st eigen vector from the diffusion maps)

In [87]:
X_train, X_test, y_train, y_test = train_test_split(data[features], data[label_column], test_size=0.33, random_state=42)

best_features, labels, features_rank = return_best_features_by_kmeans(coordinates, k)
print(f'{best_features}')
predict(X_train.iloc[:, best_features], X_test.iloc[:, best_features], y_train, y_test)

[2, 3, 7, 4]
              precision    recall  f1-score   support

           1      0.704     0.864     0.776        22
           2      0.750     0.720     0.735        25
           3      0.500     0.250     0.333         4
           5      1.000     0.500     0.667         6
           6      0.333     0.250     0.286         4
           7      0.833     1.000     0.909        10

    accuracy                          0.732        71
   macro avg      0.687     0.597     0.618        71
weighted avg      0.731     0.732     0.718        71



[0.6896551724137931,
 0.6896551724137931,
 0.6551724137931034,
 0.6428571428571429,
 0.6428571428571429]

### K-mediods classification

In [88]:
X_train, X_test, y_train, y_test = train_test_split(data[features], data[label_column], test_size=0.33, random_state=42)
k_features = k_medoids_features(coordinates, k)
print(f'{k_features}')
predict(X_train.iloc[:, k_features], X_test.iloc[:, k_features], y_train, y_test)

[3, 5, 6, 8]
              precision    recall  f1-score   support

           1      0.607     0.773     0.680        22
           2      0.696     0.640     0.667        25
           3      0.500     0.250     0.333         4
           5      1.000     0.333     0.500         6
           6      0.667     0.500     0.571         4
           7      0.769     1.000     0.870        10

    accuracy                          0.676        71
   macro avg      0.706     0.583     0.603        71
weighted avg      0.692     0.676     0.661        71



[0.5862068965517241,
 0.6551724137931034,
 0.4482758620689655,
 0.6785714285714286,
 0.6428571428571429]

## JM Distance

In [89]:
X_train, X_test, y_train, y_test = train_test_split(data[features], data[label_column], test_size=0.33, random_state=42)
# Norm
X_train_norm, X_test_norm = min_max_scaler(X_train, features, X_test)

df_dists, dist_dict = calc_dist('jm_dist', X_train_norm, y_train)
eps_type='maxmin'#mean' #or maxmin
alpha=1
vec, egs, coordinates, dataList, epsilon, ranking = (diffusion_mapping(df_dists, alpha, eps_type, 8, 1, dim=2))

### pick feature by distance from axis (0, 0)

In [90]:
best_features = return_farest_featires_from_center(k, coordinates)
print(f'{best_features}')
predict(X_train.iloc[:, best_features], X_test.iloc[:, best_features], y_train, y_test)

[0 5 8 7]
              precision    recall  f1-score   support

           1      0.655     0.864     0.745        22
           2      0.640     0.640     0.640        25
           3      0.000     0.000     0.000         4
           5      0.333     0.167     0.222         6
           6      0.800     1.000     0.889         4
           7      1.000     0.900     0.947        10

    accuracy                          0.690        71
   macro avg      0.571     0.595     0.574        71
weighted avg      0.642     0.690     0.659        71



C:\Users\chenb\anaconda3\lib\site-packages\sklearn\metrics\_classification.py:1308: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
C:\Users\chenb\anaconda3\lib\site-packages\sklearn\metrics\_classification.py:1308: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
C:\Users\chenb\anaconda3\lib\site-packages\sklearn\metrics\_classification.py:1308: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


[0.7241379310344828,
 0.6206896551724138,
 0.7241379310344828,
 0.5,
 0.6071428571428571]

### ranked columns classification (for ranking, use 1st eigen vector from the diffusion maps)

In [91]:
flat_ranking = [item for sublist in ranking for item in sublist]
ranking_idx = np.argsort(flat_ranking)
print(f'{ranking_idx}')
predict(X_train.iloc[:, ranking_idx[-k:]], X_test.iloc[:, ranking_idx[-k:]], y_train, y_test)

[7 5 8 2 1 3 6 4 0]
              precision    recall  f1-score   support

           1      0.741     0.909     0.816        22
           2      0.818     0.720     0.766        25
           3      0.667     0.500     0.571         4
           5      0.750     0.500     0.600         6
           6      0.750     0.750     0.750         4
           7      0.909     1.000     0.952        10

    accuracy                          0.789        71
   macro avg      0.772     0.730     0.743        71
weighted avg      0.789     0.789     0.782        71



[0.7586206896551724,
 0.4827586206896552,
 0.8620689655172413,
 0.7142857142857143,
 0.8214285714285714]

### ranked + K-means classification (for ranking, use 1st eigen vector from the diffusion maps)

In [92]:
X_train, X_test, y_train, y_test = train_test_split(data[features], data[label_column], test_size=0.33, random_state=42)
best_features, labels, features_rank = return_best_features_by_kmeans(coordinates, k)
print(f'{best_features}')
predict(X_train.iloc[:, best_features], X_test.iloc[:, best_features], y_train, y_test)

[7, 5, 2, 3]
              precision    recall  f1-score   support

           1      0.720     0.818     0.766        22
           2      0.704     0.760     0.731        25
           3      1.000     0.250     0.400         4
           5      1.000     0.333     0.500         6
           6      0.750     0.750     0.750         4
           7      0.833     1.000     0.909        10

    accuracy                          0.746        71
   macro avg      0.835     0.652     0.676        71
weighted avg      0.771     0.746     0.730        71



[0.5862068965517241,
 0.6551724137931034,
 0.6896551724137931,
 0.6071428571428571,
 0.6428571428571429]

### K-mediods classification

In [93]:
X_train, X_test, y_train, y_test = train_test_split(data[features], data[label_column], test_size=0.33, random_state=42)
k_features = k_medoids_features(coordinates, k)
print(f'{k_features}')
predict(X_train.iloc[:, k_features], X_test.iloc[:, k_features], y_train, y_test)

[2, 3, 4, 8]
              precision    recall  f1-score   support

           1      0.720     0.818     0.766        22
           2      0.667     0.720     0.692        25
           3      0.500     0.250     0.333         4
           5      0.667     0.333     0.444         6
           6      0.500     0.250     0.333         4
           7      0.667     0.800     0.727        10

    accuracy                          0.676        71
   macro avg      0.620     0.529     0.549        71
weighted avg      0.664     0.676     0.659        71



[0.6551724137931034,
 0.7586206896551724,
 0.6896551724137931,
 0.6785714285714286,
 0.5714285714285714]

## Hellinger Distance

In [94]:
X_train, X_test, y_train, y_test = train_test_split(data[features], data[label_column], test_size=0.33, random_state=42)
# Norm
X_train_norm, X_test_norm = min_max_scaler(X_train, features, X_test)

df_dists, dist_dict = calc_dist('hellinger_dist', X_train_norm, y_train)

eps_type='maxmin'#mean' #or maxmin
alpha=1
vec, egs, coordinates, dataList, epsilon, ranking = (diffusion_mapping(df_dists, alpha, eps_type, 8, 1, dim=2))

In [95]:
sum(np.argsort(coordinates[0]) == np.argsort(df_dists.mean(axis=1)))

1

In [96]:
np.argsort(coordinates[0])

array([2, 7, 5, 1, 3, 4, 6, 0, 8], dtype=int64)

In [97]:
#a = [1 ,4,5,6]
#b = [2 ,4,5,6]
#sum(a==b)

In [98]:
np.argsort(df_dists.mean(axis=1))

0    0
1    1
2    6
3    4
4    3
5    5
6    7
7    8
8    2
dtype: int64

### pick feature by distance from axis (0, 0)

In [99]:
best_features = return_farest_featires_from_center(k, coordinates)
print(f'{best_features}')
predict(X_train.iloc[:, best_features], X_test.iloc[:, best_features], y_train, y_test)


[0 8 7 2]
              precision    recall  f1-score   support

           1      0.731     0.864     0.792        22
           2      0.760     0.760     0.760        25
           3      0.000     0.000     0.000         4
           5      1.000     0.333     0.500         6
           6      0.571     1.000     0.727         4
           7      0.909     1.000     0.952        10

    accuracy                          0.761        71
   macro avg      0.662     0.659     0.622        71
weighted avg      0.739     0.761     0.730        71



C:\Users\chenb\anaconda3\lib\site-packages\sklearn\metrics\_classification.py:1308: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
C:\Users\chenb\anaconda3\lib\site-packages\sklearn\metrics\_classification.py:1308: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
C:\Users\chenb\anaconda3\lib\site-packages\sklearn\metrics\_classification.py:1308: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


[0.7586206896551724,
 0.6896551724137931,
 0.7241379310344828,
 0.6785714285714286,
 0.7857142857142857]

### ranked columns classification (for ranking, use 1st eigen vector from the diffusion maps)

In [100]:
flat_ranking = [item for sublist in ranking for item in sublist]
ranking_idx = np.argsort(flat_ranking)
print(f'{ranking_idx}')
predict(X_train.iloc[:, ranking_idx[-k:]], X_test.iloc[:, ranking_idx[-k:]], y_train, y_test)

[8 7 5 3 0 6 4 1 2]
              precision    recall  f1-score   support

           1      0.682     0.682     0.682        22
           2      0.654     0.680     0.667        25
           3      0.000     0.000     0.000         4
           5      1.000     0.667     0.800         6
           6      0.250     0.250     0.250         4
           7      0.643     0.900     0.750        10

    accuracy                          0.648        71
   macro avg      0.538     0.530     0.525        71
weighted avg      0.631     0.648     0.633        71



[0.6206896551724138,
 0.6206896551724138,
 0.6896551724137931,
 0.5,
 0.6071428571428571]

### ranked + K-means classification (for ranking, use 1st eigen vector from the diffusion maps)

In [101]:
X_train, X_test, y_train, y_test = train_test_split(data[features], data[label_column], test_size=0.33, random_state=42)

best_features, labels, features_rank = return_best_features_by_kmeans(coordinates, k)
print(f'{best_features}')
predict(X_train.iloc[:, best_features], X_test.iloc[:, best_features], y_train, y_test)

[2, 7, 5, 8]
              precision    recall  f1-score   support

           1      0.692     0.818     0.750        22
           2      0.654     0.680     0.667        25
           3      1.000     0.250     0.400         4
           5      1.000     0.333     0.500         6
           6      0.667     1.000     0.800         4
           7      1.000     1.000     1.000        10

    accuracy                          0.732        71
   macro avg      0.835     0.680     0.686        71
weighted avg      0.764     0.732     0.718        71



[0.5172413793103449,
 0.6206896551724138,
 0.4827586206896552,
 0.39285714285714285,
 0.6071428571428571]

### K-mediods classification

In [102]:
X_train, X_test, y_train, y_test = train_test_split(data[features], data[label_column], test_size=0.33, random_state=42)

k_features = k_medoids_features(coordinates, k)
print(f'{k_features}')
predict(X_train.iloc[:, k_features], X_test.iloc[:, k_features], y_train, y_test)

[0, 1, 4, 5]
              precision    recall  f1-score   support

           1      0.593     0.727     0.653        22
           2      0.577     0.600     0.588        25
           3      0.000     0.000     0.000         4
           5      0.500     0.167     0.250         6
           6      0.667     0.500     0.571         4
           7      0.667     0.800     0.727        10

    accuracy                          0.592        71
   macro avg      0.500     0.466     0.465        71
weighted avg      0.560     0.592     0.565        71



[0.6896551724137931,
 0.6551724137931034,
 0.6551724137931034,
 0.75,
 0.4642857142857143]